
##### OBJETIVO: interpolar las mediciones AOD de cada estacion 
##### AERONET a la longitud de onda proporcionada por MAIAC-MODIS (550).

Funcion para estimar el AOD550 a partir del coeficiente de Angstroment


In [1]:
# Library
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import scipy.stats
from sklearn.metrics import mean_squared_error
from math import sqrt
import os
from os import listdir
from datetime import datetime
import pandas as pd
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression

In [3]:
# Directorio donde se encuentra el archivo con los datos de AOD
path = "D:/Josefina/paper_git/paper_maiac/datasets/V02/aeronet/datasets_interp_s_L02/original/"
path_write="D:/Josefina/paper_git/paper_maiac/datasets/V02/aeronet/datasets_interp_s_L02/Latam/"

In [4]:
os.chdir(path)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'D:/Josefina/paper_git/paper_maiac/datasets/V02/aeronet/datasets_interp_s_L02/original/'

In [ ]:
def interpolate(path, path_write):
    os.makedirs(path_write, exist_ok=True)

    archivos = [f for f in os.listdir(path) if f.endswith(".csv")]

    for i, archivo in enumerate(archivos, start=1):
        print(i)

        # Leer archivo
        data = pd.read_csv(
            os.path.join(path, archivo),
            na_values=-999
        )

        # Crear columna fecha
        data["date"] = pd.to_datetime(
            data["Date.dd.mm.yyyy."] + " " + data["Time.hh.mm.ss."],
            format="%d:%m:%Y %H:%M",
            utc=True
        )

        # Nombre base del archivo que corresponde a las ciudades
        name = archivo[:12]

        # Calcular AOD 550 nm con las distintas longitudes de onda
        data["aod_550"] = data["AOD_500nm"] * (
            (550 / 500) ** (-data["X440.675_Angstrom_Exponent"])
        )

        # Dataset final
        df = data[["date", "aod_550"]]
        # Guardar
        #output = os.path.join(path_write, f"{name}_interp-a.csv")
        #df.to_csv(output, index=False)

        return df


Versión 2: interpolación cuadrática (Lagrange)

In [ ]:
def interpol_cuad(x, x0, y0, x1, y1, x2, y2):
    a = ((x - x1) * (x - x2)) / ((x0 - x1) * (x0 - x2))
    b = ((x - x0) * (x - x2)) / ((x1 - x0) * (x1 - x2))
    c = ((x - x0) * (x - x1)) / ((x2 - x0) * (x2 - x1))
    return y0 * a + y1 * b + y2 * c


In [ ]:
def interpolate_s(path, path_write):
    os.makedirs(path_write, exist_ok=True)

    archivos = [f for f in os.listdir(path) if f.endswith(".csv")]

    for i, archivo in enumerate(archivos, start=1):
        print(i)

        data = pd.read_csv(
            os.path.join(path, archivo),
            na_values=-999
        )

        # Fecha
        date = pd.to_datetime(
            data["Date.dd.mm.yyyy."] + " " + data["Time.hh.mm.ss."],
            format="%d:%m:%Y %H:%M",
            utc=True
        )

        name = archivo[:12]

        # Valor objetivo
        x = np.log(550)

        # Puntos base
        x0 = np.log(440)
        y0 = np.log(data["AOD_440nm"])

        x1 = np.log(500)
        y1 = np.log(data["AOD_500nm"])

        # --- Primera interpolacion (675)
        x2 = np.log(675)
        y2 = np.log(data["AOD_675nm"])
        aod_550_1 = np.exp(interpol_cuad(x, x0, y0, x1, y1, x2, y2))

        # --- Segunda interpolacion (870)
        x2 = np.log(870)
        y2 = np.log(data["AOD_870nm"])
        aod_550_2 = np.exp(interpol_cuad(x, x0, y0, x1, y1, x2, y2))

        # --- Tercera interpolacion (1020)
        x2 = np.log(1020)
        y2 = np.log(data["AOD_1020nm"])
        aod_550_3 = np.exp(interpol_cuad(x, x0, y0, x1, y1, x2, y2))

        # DataFrame final
        df = pd.DataFrame({
            "date": date,
            "aod_550": aod_550_1,
            "aod_550_2": aod_550_2,
            "aod_550_3": aod_550_3
        })

        df["aod_550_mod"] = df[["aod_550", "aod_550_2", "aod_550_3"]].mean(axis=1)

        output = os.path.join(path_write, f"{name}_interp-s.csv")
        df.to_csv(output, index=False)


Ejemplo para llamar  a las funciones

In [ ]:
interpolate(
    path="D:/Josefina/Proyectos/aeronet/datos/AERONET_02112023_L02/original",
    path_write="D:/Josefina/Proyectos/aeronet/datos/AERONET_02112023_L02/interpolate"
)

interpolate_s(
    path="D:/Josefina/Proyectos/aeronet/datos/AERONET_02112023_L02/original",
    path_write="D:/Josefina/Proyectos/aeronet/datos/AERONET_02112023_L02/interpolate"
)
